In [1]:
import pandas as pd
import numpy as np
import requests
import yaml
import time
from random import shuffle
import json
from pymongo import MongoClient
from collections import Counter

In [184]:
def create_pandas_df_from_json(path):
    '''
    INPUT: filepath string
    OUTPUT: pandas database
    '''
    return pd.read_json(file_path, lines=True)

def is_food(item):
    '''
    INPUT: cell from pandas dataframe
    OUTPUT: boolean
    '''
    restaurants_and_related_categories = ['Restaurants', 'Italian','Food', 'Bars','Fast Food', 'Coffee & Tea', 'Sandwiches']
    if len(set(restaurants_and_related_categories) & set(item)) >= 1:
        return True
    else:
        return False
    
def flatten_dict(row):
    out = {}
    for key, value in row.items():
        if type(value) != dict:
            out[key] = value
        else:
            sub_key = key
            for k, v in value.items():
                out[sub_key + "|" + k] = v
    return out

def make_exists_function(key):
    def get_key_if_exists(row):
        if key in row:
            return row[key]
        else:
            return "N/A"
    return get_key_if_exists

def add_restaurant_count_column(dataframe):
    restaurant_frequency = dataframe.groupby(['name']).count().sort_values('address', ascending=False)

    restaurant_frequency = pd.DataFrame(restaurant_frequency['address'])

    restaurant_frequency.columns = ['count']

    restaurant_frequency['name'] = restaurant_frequency.index

    restaurant_frequency = restaurant_frequency[['name', 'count']]

    return previously_open_US_restaurants.merge(restaurant_frequency, how='left', left_on='name', right_on='name')

def closed_on_google(row):
    try:
        return row[0]['permanently_closed']
    except:
        return False

In [191]:
file_path = '~/g/projects/yelp/dataset/business.json'
yelp_business_data = create_pandas_df_from_json(file_path)

#filters businesses that were open when this dataset was published Jan. 2018
open_businesses = yelp_business_data[yelp_business_data['is_open'] == 1]

#creates column that says if business is restaurant and creates df of just open restaurants
open_businesses['is_food'] = open_businesses['categories'].apply(is_food)
open_restaurants = open_businesses[open_businesses['is_food'] == True]

#creates column that says if business is in USA and creates df of just
#restaurants open in the US as of January 2018
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA",
      "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD",
      "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ",
      "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC",
      "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
open_restaurants['in_US'] = open_restaurants['state'].isin(states)
previously_open_US_restaurants = open_restaurants[open_restaurants['in_US'] == True]

#creates dummy columns for 
previously_open_US_restaurants['flat_attributes'] = previously_open_US_restaurants['attributes'].apply(flatten_dict)
all_attributes = []
for row in previously_open_US_restaurants['flat_attributes']:
    all_attributes.extend(row.keys())
unique_attributes = list(dict(Counter(all_attributes).most_common(50)).keys())

for key in unique_attributes:
    previously_open_US_restaurants['has_'+key] = previously_open_US_restaurants['flat_attributes'].apply(lambda x: key in x)
    
    f = make_exists_function(key)
    previously_open_US_restaurants[key + ' value:'] = previously_open_US_restaurants['flat_attributes'].apply(f)
    
all_categories = []
[all_categories.extend(item) for item in list(previously_open_US_restaurants['categories'])]

most_common_categories = list(dict(Counter(all_categories).most_common(50)).keys())

for key in most_common_categories:
    previously_open_US_restaurants[f"{key}_true"] = previously_open_US_restaurants['categories'].apply(lambda x: key in x)

client = MongoClient('mongodb://localhost:27017/')
restaurants = client['restaurants']
google_places = restaurants['google_places']
start_time = time.time()

google_df = pd.DataFrame(list(google_places.find()))

google_df = google_df[['queried_name', 'yelp_business_id', 'results']]

google_df['closed_on_google'] = google_df['results'].apply(closed_on_google)

restaurants_with_google_data = previously_open_US_restaurants.merge(google_df, how='inner', left_on='business_id', right_on='yelp_business_id')

#removes rows without any matching data from Google
restaurants_with_google_data = restaurants_with_google_data[restaurants_with_google_data['results'].map(len) > 0]

/Users/ElliottC/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/ElliottC/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/ElliottC/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pand

In [189]:
pd.DataFrame(list(google_places.find()))

,_id,error_message,html_attributions,next_page_token,queried_latitude,queried_longitude,queried_name,results,status,yelp_business_id
0,5af216f10625fc0e86a89e7d,NaN,[],NaN,40.327322,-80.042015,"Au ""Grottens"" Cafe & Catering","[{'geometry': {'location': {'lat': 40.327322, ...",OK,KE7Uq1iXOQVhsPIzE800Aw
1,5af2171a0625fc0e86a89e7f,NaN,[],NaN,40.327322,-80.042015,"Au ""Grottens"" Cafe & Catering","[{'geometry': {'location': {'lat': 40.327322, ...",OK,KE7Uq1iXOQVhsPIzE800Aw
2,5af2171b0625fc0e86a89e80,NaN,[],NaN,41.154095,-81.335745,Rockne's,"[{'geometry': {'location': {'lat': 41.1538296,...",OK,j6wN7ZFx5bl2t4KfLaAbJQ
3,5af2171c0625fc0e86a89e81,NaN,[],NaN,33.488553,-111.961302,Smoothie King,"[{'geometry': {'location': {'lat': 33.4885461,...",OK,UiLrSPbrlY94Gwyymy1-hw
4,5af2171c0625fc0e86a89e82,NaN,[],NaN,41.581839,-81.204172,Element 41,"[{'geometry': {'location': {'lat': 41.5818233,...",OK,4dWhd2lHIELpI0XTvs8_8A
5,5af2171d0625fc0e86a89e83,NaN,[],NaN,33.451794,-112.394352,Starbucks,"[{'geometry': {'location': {'lat': 33.4516319,...",OK,1nno5CjwR2JwUHMGZp2AzA
6,5af2171e0625fc0e86a89e84,NaN,[],NaN,33.436473,-112.058373,Petes Fish & Chips,"[{'geometry': {'location': {'lat': 33.4364836,...",OK,_MeGxn_23d51eOyk-0XuHg
7,5af2171e0625fc0e86a89e85,This service requires an API key.,[],NaN,36.158602,-115.282371,Jack-In-the-Box #7239,[],REQUEST_DENIED,1gjhtfYxlN-0xjS9Xdh66g
8,5af2171e0625fc0e86a89e86,NaN,[],NaN,41.490803,-81.672676,Randys Pickles,[],ZERO_RESULTS,LzVfKUFHb597LHwheowNKg
9,5af2171f0625fc0e86a89e87,NaN,[],NaN,33.378095,-111.890813,Jz Market,"[{'geometry': {'location': {'lat': 33.378095, ...",OK,G1hRBgNP116ET-a2TBF3rw
